##Install Important Libraries

In [ ]:
!pip install bs4

In [ ]:
!pip install jmd_imagescraper

In [1]:
#export

# scraping
from pathlib import Path
from typing import Union
from enum import Enum
import re
import requests
import json
import time
from bs4 import BeautifulSoup
import uuid

# other
from PIL import Image as PImage
from IPython.display import display
import pandas as pd
from fastprogress.fastprogress import progress_bar

In [17]:
from google import colab
colab.drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
!pip install jmd_imagescraper

##Download UNPLASH images

In [19]:
from pathlib import Path
lik =Path().cwd()/"https://images.unsplash.com/photo-1607596798924-ae8a3d31090b?ixid=MnwyNjAzMTR8MHwxfHNlYXJjaHwxfHxhcmFiJTIwbWFufGVufDB8fHx8MTYzMTUxODg0Ng&ixlib=rb-1.2.1"

from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

duckduckgo_search(lik, "image", "image", max_results=1)
# duckduckgo_search(lik, "human face", "human faces", max_results=1000)
# duckduckgo_search(lik, "Aerial Images", "Aerial Images", max_results=1000)
# duckduckgo_search(lik, "Building", "Building", max_results=5000)
# duckduckgo_search(lik, "Photography", "Photography", max_results=5000)
# duckduckgo_search(lik, "Animals", "Animals", max_results=5000)

Duckduckgo search: image


[PosixPath('/content/https:/images.unsplash.com/photo-1607596798924-ae8a3d31090b?ixid=MnwyNjAzMTR8MHwxfHNlYXJjaHwxfHxhcmFiJTIwbWFufGVufDB8fHx8MTYzMTUxODg0Ng&ixlib=rb-1.2.1/image/004_2d1ae533.jpg')]

##Defining The Path

In [ ]:
#export
def rmtree(path: Union[str, Path]):
    '''Recursively delete a directory tree'''
    path = Path(path); assert path.is_dir()
    for p in reversed(list(path.glob('**/*'))):
        if p.is_file():  p.unlink()
        elif p.is_dir(): p.rmdir()
    path.rmdir()

In [9]:

def download_urls(path: Union[str, Path], links: list, uuid_names: bool=True) -> list:
  '''Downloads urls to the given path. Returns a list of Path objects for files downloaded to disc.'''
  if(len(links) == 0):
    print("Nothing to download!"); return

  path = Path(path)
  path.mkdir(parents=True, exist_ok=True)

  print("Downloading results into", path)
  pbar = progress_bar(links)
  pbar.comment = 'Images downloaded'

  i = 1
  mk_uniq = lambda : '_' + str(uuid.uuid4())[:8] if uuid_names else ''
  mk_fp = lambda x: path/(str(x).zfill(3) + mk_uniq() + ".jpg")
  is_file = lambda x: len(list(path.glob(str(x).zfill(3) + '*.jpg'))) > 0
    
  while is_file(i): i += 1 # don't overwrite previous searches
  
  results = []
    
  for link in pbar:
      try:
        resp = requests.get(link)
        fp = mk_fp(i)
        fp.write_bytes(resp.content)

        try:
          img = PImage.open(fp)
          img.verify()
          img.close()
          results.append(Path(fp))
        except Exception as e:
          # print(e)
          print(fp, "is invalid")
          fp.unlink()
      except Exception as e:
        # print(e)
        print("Exception occured while retrieving", link)
        
      i += 1

  return results

In [10]:
params = {
    "max_results": 1,
     
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.All,
    
}

links = duckduckgo_scrape_urls("Unplash", **params)
links

['https://tse2.mm.bing.net/th?id=OIP.H5T5wthEShwVZGR3ME4gCgHaE8&pid=Api']

In [ ]:
lik = Path.cwd()/"gdrive/My Drive/colab/Unsplash"
download_urls(lik/"human face", links)


[PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/655_27dc46d6.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/656_e2a074fd.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/657_a0fb22aa.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/658_6f0a9a49.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/659_7ed124b2.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/660_f5a17b45.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/661_76f3f1e0.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/662_028e683f.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/663_f8322619.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/664_f7b38296.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/665_d2bc861d.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/666_14dca1a9.jpg'),
 Pos

##Changing params across multiple searches

In [ ]:
# If you're going to override default params across multiple searches you can use a 
# dictionary like this (so you can change search params for the entire dataset once).

params = {
    "max_results": 12000,             
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.Square,
    
}

duckduckgo_search(lik, "human face", "human face", **params)
# duckduckgo_search(lik,  "Aerial Images", "Aerial Images", **params)
# duckduckgo_search(lik,  "Beach", "Beach", **params)
# duckduckgo_search(lik,  "Building", "Building", **params)
# duckduckgo_search(lik,  "Animal", "Animal", **params)
# duckduckgo_search(lik,  "Photography", "Photography", **params)

Duckduckgo search: human face


Exception occured while retrieving https://tse1.mm.bing.net/th?id=OIP.vEn3r8AbHNCK_h_k_rLlXAHaHa&pid=Api


[PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/001_e4d1134b.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/002_b503f38e.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/003_4d211908.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/004_63906623.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/005_bdd6a5f9.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/006_b3bbcc91.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/007_a8df413b.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/008_fa990e7a.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/009_882c905e.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/010_357e2c90.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/011_47ea0e37.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/human face/012_60dcb1c0.jpg'),
 Pos

In [15]:
# If you're going to override default params across multiple searches you can use a 
# dictionary like this (so you can change search params for the entire dataset once).

params = {
    "max_results": 1,             
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.Square,
    
}
duckduckgo_search(lik,  "image", "image", **params)
# duckduckgo_search(lik, "People", "People", **params)
# duckduckgo_search(lik,  "Background", "Background", **params)
# duckduckgo_search(lik,  "Beach", "Beach", **params)
# duckduckgo_search(lik,  "Fall", "Fall", **params)
# duckduckgo_search(lik,  "Sunset", "Sunset", **params)
# duckduckgo_search(lik,  "Building", "Building", **params)

# duckduckgo_search(lik,  "Photography", "Photography", **params)
# # duckduckgo_search(root,  "woman", "woman", **params)duckduckgo_search(root,  "face", "human faces", **params)
# duckduckgo_search(lik,  "Cloud", "Cloud", **params)
# duckduckgo_search(lik,  "Road", "Road", **params)
# duckduckgo_search(lik,  "lion", "lion", **params)
# duckduckgo_search(lik,  "animal", "animal", **params)
# # duckduckgo_search(root,  "priest", "priest", **params)
# # duckduckgo_search(root,  "athlete", "athlete", **params)
# # duckduckgo_search(root,  "woman", "woman", **params)

Duckduckgo search: image


[PosixPath('/content/https:/images.unsplash.com/photo-1607596798924-ae8a3d31090b?ixid=MnwyNjAzMTR8MHwxfHNlYXJjaHwxfHxhcmFiJTIwbWFufGVufDB8fHx8MTYzMTUxODg0Ng&ixlib=rb-1.2.1/image/003_fc09dbaf.jpg')]

##Displaying the image cleaner

In [20]:
from jmd_imagescraper.imagecleaner import *
display_image_cleaner(lik)

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…